In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

## L

In [ ]:
%%time
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import re
import nltk
import json
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
from sklearn.metrics import average_precision_score,roc_auc_score, roc_curve, precision_recall_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
np.random.seed(42)
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import ToktokTokenizer
import nltk, string, re, spacy,unicodedata, random
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')
### Pretrained Word Embeddings

folder_path = "/content/drive/MyDrive/model_imp_whole/"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
CPU times: user 1.42 s, sys: 350 ms, total: 1.77 s
Wall time: 3.52 s


## BE CODEMIXED Dataset


In [ ]:
%%time
train_data =  pd.read_excel('/content/drive/MyDrive/model_imp_whole/train.xlsx') 
valid_data = pd.read_excel('/content/drive/MyDrive/model_imp_whole/val.xlsx')
test_data = pd.read_excel('/content/drive/MyDrive/model_imp_whole/test.xlsx')

CPU times: user 300 ms, sys: 19.4 ms, total: 319 ms
Wall time: 1.34 s


In [ ]:
valid_data.head(2)

,Label,Text
0,Anger,Jat tai to beiman katar jat.ki6u bol tora
1,Sadness,Porki - Amader r koto baar duchbe 😭 😭 😭


In [ ]:
train_data['Label'].value_counts()

Joy         715
Sadness     394
Neutral     389
Disgust     311
Anger       146
Surprise    122
Fear         29
Name: Label, dtype: int64

In [ ]:
train_data['enc_label'] = train_data['Label'].replace({'Neutral' :6 ,'Joy': 0,'Disgust':  3,'Anger' :4,
                                                       'Sadness': 1,'Surprise':2,'Fear': 5})
valid_data['enc_label'] = valid_data['Label'].replace({'Neutral' :6 ,'Joy': 0,'Disgust':  3,'Anger' :4,
                                                       'Sadness': 1,'Surprise':2,'Fear': 5})

In [ ]:
train_data.head(1)

,Label,Text,enc_label
0,Disgust,Dakhachish ai to tui svf are chala..to jeet d...,3


In [ ]:
test_data.head(1)

,Unnamed: 0,Text
0,0,Depression niye beche thakar cheye moira jaoya...


In [ ]:
# train_data['Text'].tolist()

In [ ]:
train_data['enc_label'].value_counts()

0    715
1    394
6    389
3    311
4    146
2    122
5     29
Name: enc_label, dtype: int64

In [ ]:
valid_data['Label'].value_counts()

Joy         153
Disgust      78
Neutral      78
Sadness      77
Surprise     32
Anger        30
Fear          3
Name: Label, dtype: int64

In [ ]:
print("Size of the training data ==> ", train_data.shape)
print("Size of the validation data ==>", valid_data.shape)
print("Size of the Test data ==>", test_data.shape)

Size of the training data ==>  (2106, 3)
Size of the validation data ==> (451, 3)
Size of the Test data ==> (450, 2)


## One Hot Encoding

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [ ]:
encoder=OneHotEncoder(sparse=False)

encoded_labels = pd.DataFrame (encoder.fit_transform(train_data[['Label']]))

encoded_labels .columns = encoder.get_feature_names(['Label'])

train_data= pd.concat([train_data, encoded_labels ], axis=1)

In [ ]:
train_data = train_data.rename(columns={'Label_Neutral' :'Neutral' ,'Label_Joy': 'Joy',
                                        'Label_Disgust':  'Disgust','Label_Anger' :'Anger',
                                        'Label_Sadness': 'Sadness',
                                        'Label_Surprise':'Surprise','Label_Fear': 'Fear'
                                      
                                      })
train_data.head(1)

,Label,Text,enc_label,Anger,Disgust,Fear,Joy,Neutral,Sadness,Surprise
0,Disgust,Dakhachish ai to tui svf are chala..to jeet d...,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0


## Cleaning

In [ ]:
'''
Text Cleaning
'''
def text_cleaning(row):
  # to remove HTML tags
  text = BeautifulSoup(row, 'html.parser').get_text()
  d = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE) #This line is for removing url
  post = d.replace('\n', '')
  post = post.replace('—', ' ')
  # to remove accented characters
  new_text = unicodedata.normalize('NFKD', post).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  # to remove special characters and numbers
  # define the pattern to keep
  #pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 
  #text = re.sub(pat, '', new_text)
  #to remove punctuation
  text = ''.join([c for c in text if c not in string.punctuation])
  # to remove special characters
  pattern = r'^\s*|\s\s*'
  text = re.sub(pattern, ' ', text).strip()
  # convert into lower case
  text = text.lower() 
  # Stopword Removing
  tokenizer = ToktokTokenizer()
  # convert sentence into token of words
  tokens = tokenizer.tokenize(text)
  tokens = [token.strip() for token in tokens]
  # check in lowercase 
  #t = [token for token in tokens if token.lower() not in stopword_list]
  #text = ' '.join(t) 
  # Stemming
  #t = [stemming.stem(token) for token in tokens ]
  #text = ' '.join(t) 

  return text



In [ ]:
def get_becodemixed_stop_words():
    #swdf1 = pd.read_csv("/content/drive/MyDrive/automated_tamil_text_classification-master/data/TamilNLP_TamilStopWords.txt",  header=None) 
    swdf2 = pd.read_csv("/content/drive/MyDrive/model_imp_whole/custom_becodemixed_stopwords.txt",  header=None) 
    #sw1 = swdf1[0].tolist()
    sw2 = swdf2[0].tolist()
    #tamil_stop_words = list(set(sw1 + sw2))
    becodemixed_stop_words = list(sw2)
    return becodemixed_stop_words

In [ ]:
#removing URLs and Punctuations
#train_data['cleaned'] = train_data['tweet'].apply(remove_url)
#Removing punctuations


becodemixed_stop_words = get_becodemixed_stop_words()
valid_data['Texts'] = valid_data['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (becodemixed_stop_words)]))
train_data['Texts'] = train_data['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (becodemixed_stop_words)]))
test_data['Texts'] = test_data['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (becodemixed_stop_words)]))

train_data['cleaned'] = train_data['Texts'].apply(text_cleaning)
valid_data['cleaned'] = valid_data['Texts'].apply(text_cleaning)
test_data['cleaned'] = test_data['Texts'].apply(text_cleaning)


## Data samples after cleaning
print("Original Data:===\n",train_data.Text[0],"\nCleaned Data:===\n",train_data.cleaned[0],)

Original Data:===
  Dakhachish ai to tui svf are chala..to jeet dar collection to fake blban e to ... sir manush boka noi 
Cleaned Data:===
 dakhachish ai tui svf are chalato jeet dar collection fake blban sir manush boka noi


In [ ]:
valid_data['cleaned'].shape

(451,)

##ML

In [ ]:
def print_metrices(pred,true):
    print(confusion_matrix(true,pred))
    print(classification_report(true,pred,))
    print("Accuracy : ",accuracy_score(pred,true))
    print("Precison : ",precision_score(pred,true, average = 'weighted'))
    print("Recall : ",recall_score(pred,true,  average = 'weighted'))
    print("F1 : ",f1_score(pred,true,  average = 'weighted'))

In [ ]:
## TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics

tfidfvectorizer = TfidfVectorizer(max_features=40000,ngram_range=(1,2),tokenizer=lambda x: x.split())
tfidfvectorised = tfidfvectorizer.fit_transform(train_data['cleaned'])
tfidfvectorized_train = tfidfvectorizer.transform(train_data['cleaned'])
tfidfvectorized_valid = tfidfvectorizer.transform(valid_data['cleaned'])
tfidfvectorized_test = tfidfvectorizer.transform(test_data['cleaned'])

print("Feature Size: ", tfidfvectorized_train.shape[1])


Feature Size:  28641


In [ ]:
tfidfvectorized_valid.shape

(451, 28641)

In [ ]:
valid_data['Label'].value_counts()

Joy         153
Disgust      78
Neutral      78
Sadness      77
Surprise     32
Anger        30
Fear          3
Name: Label, dtype: int64

In [ ]:
actual_labels = pd.read_excel('/content/drive/MyDrive/model_imp_whole/test_with_label_forwhole.xlsx')

actual_labels 

,label,text
0,Sadness,Depression niye beche thakar cheye moira jaoya...
1,Joy,Dev is Dev ! ! Dev ke sobi bhalo lage 😘 😘 😍
2,Disgust,Ebe ke gopolo ole wa ` ` stoked '' 😂 😂 😂 😂 😂 😂...
3,Anger,Han je desh e ekta 14th century r mosque n...
4,Joy,"Lol , thake gape iye , kere kgane o re plokil..."
...,...,...
445,Joy,"Eid mane akashe notun chad , Eid mane notun ki..."
446,Disgust,Eber tahola k sobai tel lagai ... . Ajob s...
447,Joy,Mojar chilo . koto manusher jhogra hoye gelo ...
448,Sadness,apnake ekhon abp ananda te dekte paina keno


In [ ]:
# labels = ['Neutral','Joy','Disgust','Anger','Sadness','Surprise','Fear']
labels = ['Joy','Sadness','Surprise','Disgust','Anger','Fear','Neutral']
                                                       
def actual_label(row):
    return labels[row]

### Logistic Regression

In [ ]:

lr = LogisticRegression(class_weight='balanced',C =10)
lr.fit(tfidfvectorized_train,train_data['enc_label'])
y_pred = lr.predict(tfidfvectorized_valid)
print_metrices(y_pred,valid_data['enc_label'])

[[103  20   5   5   3   0  17]
 [ 23  36   5   5   1   0   7]
 [ 13   1   6   9   0   0   3]
 [ 19  14   2  28   5   0  10]
 [  5   5   0   9   5   0   6]
 [  2   1   0   0   0   0   0]
 [ 25   8   0   9   0   0  36]]
              precision    recall  f1-score   support

           0       0.54      0.67      0.60       153
           1       0.42      0.47      0.44        77
           2       0.33      0.19      0.24        32
           3       0.43      0.36      0.39        78
           4       0.36      0.17      0.23        30
           5       0.00      0.00      0.00         3
           6       0.46      0.46      0.46        78

    accuracy                           0.47       451
   macro avg       0.36      0.33      0.34       451
weighted avg       0.46      0.47      0.46       451

Accuracy :  0.4745011086474501
Precison :  0.5169666554334613
Recall :  0.4745011086474501
F1 :  0.49018675820224034


In [ ]:
test_pred = lr.predict(tfidfvectorized_test).tolist()
test_data['id'] = test_data.index + 1

In [ ]:
df = pd.DataFrame(test_data['id'])
df['text'] = test_data['Text']
df['label'] = test_pred
df['label'] = df['label'].apply(actual_label)
df

,id,text,label
0,1,Depression niye beche thakar cheye moira jaoya...,Sadness
1,2,Dev is Dev ! ! Dev ke sobi bhalo lage 😘 😘 😍,Joy
2,3,Ebe ke gopolo ole wa ` ` stoked '' 😂 😂 😂 😂 😂 😂...,Joy
3,4,Han je desh e ekta 14th century r mosque n...,Disgust
4,5,"Lol , thake gape iye , kere kgane o re plokil...",Neutral
...,...,...,...
445,446,"Eid mane akashe notun chad , Eid mane notun ki...",Joy
446,447,Eber tahola k sobai tel lagai ... . Ajob s...,Disgust
447,448,Mojar chilo . koto manusher jhogra hoye gelo ...,Joy
448,449,apnake ekhon abp ananda te dekte paina keno,Joy


In [ ]:
df['label'].value_counts()

Joy         194
Sadness      85
Neutral      73
Disgust      69
Anger        16
Surprise     13
Name: label, dtype: int64

In [ ]:
df.to_csv(folder_path +'becodemixed_logisticRegression.tsv',sep = '\t',index = False)

In [ ]:
print_metrices(actual_labels['label'],df['label'])

[[  8   2   0   2   3   1   0]
 [  7  30   1   8  10   7   6]
 [  0   0   0   0   0   0   0]
 [  7  14   2 117  24  20  10]
 [  2   6   0  21  31  11   2]
 [  9   9   0  11  11  43   2]
 [  3   1   0   2   2   0   5]]
              precision    recall  f1-score   support

       Anger       0.22      0.50      0.31        16
     Disgust       0.48      0.43      0.46        69
        Fear       0.00      0.00      0.00         0
         Joy       0.73      0.60      0.66       194
     Neutral       0.38      0.42      0.40        73
     Sadness       0.52      0.51      0.51        85
    Surprise       0.20      0.38      0.26        13

    accuracy                           0.52       450
   macro avg       0.36      0.41      0.37       450
weighted avg       0.56      0.52      0.54       450

Accuracy :  0.52
Precison :  0.5056654615866533
Recall :  0.52
F1 :  0.5044771004283793


## DT

In [ ]:
dt = DecisionTreeClassifier(class_weight='balanced')
dt.fit(tfidfvectorized_train,train_data['enc_label'])
y_pred = dt.predict(tfidfvectorized_valid)
print_metrices(y_pred,valid_data['enc_label'])

[[54 20  7 14  9  0 49]
 [14 28  4  8  3  0 20]
 [ 4  2  8  6  4  0  8]
 [15 13  8 14  5  0 23]
 [ 2  5  3  2  4  0 14]
 [ 0  2  0  0  0  0  1]
 [14  8  0 15  9  0 32]]
              precision    recall  f1-score   support

           0       0.52      0.35      0.42       153
           1       0.36      0.36      0.36        77
           2       0.27      0.25      0.26        32
           3       0.24      0.18      0.20        78
           4       0.12      0.13      0.12        30
           5       0.00      0.00      0.00         3
           6       0.22      0.41      0.28        78

    accuracy                           0.31       451
   macro avg       0.25      0.24      0.24       451
weighted avg       0.34      0.31      0.32       451

Accuracy :  0.31042128603104213
Precison :  0.327377709026333
Recall :  0.31042128603104213
F1 :  0.3048723572928571


In [ ]:
test_pred = dt.predict(tfidfvectorized_test).tolist()
test_data['id'] = test_data.index + 1

In [ ]:
df = pd.DataFrame(test_data['id'])
df['text'] = test_data['Text']
df['label'] = test_pred
df['label'] = df['label'].apply(actual_label)
df

,id,text,label
0,1,Depression niye beche thakar cheye moira jaoya...,Neutral
1,2,Dev is Dev ! ! Dev ke sobi bhalo lage 😘 😘 😍,Surprise
2,3,Ebe ke gopolo ole wa ` ` stoked '' 😂 😂 😂 😂 😂 😂...,Joy
3,4,Han je desh e ekta 14th century r mosque n...,Disgust
4,5,"Lol , thake gape iye , kere kgane o re plokil...",Sadness
...,...,...,...
445,446,"Eid mane akashe notun chad , Eid mane notun ki...",Joy
446,447,Eber tahola k sobai tel lagai ... . Ajob s...,Anger
447,448,Mojar chilo . koto manusher jhogra hoye gelo ...,Joy
448,449,apnake ekhon abp ananda te dekte paina keno,Joy


In [ ]:
print_metrices(actual_labels['label'],df['label'])

[[ 6  7  0 10  4  8  1]
 [ 7 18  0 19 15  4  8]
 [ 1  0  0  0  0  0  0]
 [ 5  9  1 50 16 15  8]
 [ 8 20  1 45 34 24  4]
 [ 8  5  1 20  7 26  1]
 [ 1  3  0 17  5  5  3]]
              precision    recall  f1-score   support

       Anger       0.17      0.17      0.17        36
     Disgust       0.29      0.25      0.27        71
        Fear       0.00      0.00      0.00         1
         Joy       0.31      0.48      0.38       104
     Neutral       0.42      0.25      0.31       136
     Sadness       0.32      0.38      0.35        68
    Surprise       0.12      0.09      0.10        34

    accuracy                           0.30       450
   macro avg       0.23      0.23      0.23       450
weighted avg       0.31      0.30      0.30       450

Accuracy :  0.30444444444444446
Precison :  0.3398466212053619
Recall :  0.30444444444444446
F1 :  0.3108626664347453


### SVM

In [ ]:
# C : float, default=1.0
#     Regularization parameter. The strength of the regularization is inversely proportional to C. Must be strictly positive
# The "balanced" mode uses the values of y to automatically adjust weights inversely proportional to class frequencies in the input data as n_samples / (n_classes * np.bincount(y)).

In [ ]:
## SVM
from sklearn.svm import SVC
svc =  SVC(C= 7, kernel='rbf' ,random_state=265,class_weight='balanced')
svc.fit(tfidfvectorized_train,train_data['enc_label'])
y_pred = svc.predict(tfidfvectorized_valid)
print_metrices(y_pred,valid_data['enc_label'])

[[137   6   0   1   0   0   9]
 [ 50  20   0   3   0   0   4]
 [ 25   1   1   4   0   0   1]
 [ 53  10   0  12   0   0   3]
 [ 23   3   0   2   2   0   0]
 [  2   1   0   0   0   0   0]
 [ 50   3   0   2   0   0  23]]
              precision    recall  f1-score   support

           0       0.40      0.90      0.56       153
           1       0.45      0.26      0.33        77
           2       1.00      0.03      0.06        32
           3       0.50      0.15      0.24        78
           4       1.00      0.07      0.12        30
           5       0.00      0.00      0.00         3
           6       0.57      0.29      0.39        78

    accuracy                           0.43       451
   macro avg       0.56      0.24      0.24       451
weighted avg       0.54      0.43      0.37       451

Accuracy :  0.43237250554323725
Precison :  0.7350882010942986
Recall :  0.43237250554323725
F1 :  0.49902850778259955


In [ ]:
test_pred = svc.predict(tfidfvectorized_test).tolist()
test_data['id'] = test_data.index + 1

In [ ]:
df = pd.DataFrame(test_data['id'])
df['text'] = test_data['Text']
df['label'] = test_pred
df['label'] = df['label'].apply(actual_label)
df

,id,text,label
0,1,Depression niye beche thakar cheye moira jaoya...,Joy
1,2,Dev is Dev ! ! Dev ke sobi bhalo lage 😘 😘 😍,Joy
2,3,Ebe ke gopolo ole wa ` ` stoked '' 😂 😂 😂 😂 😂 😂...,Joy
3,4,Han je desh e ekta 14th century r mosque n...,Disgust
4,5,"Lol , thake gape iye , kere kgane o re plokil...",Joy
...,...,...,...
445,446,"Eid mane akashe notun chad , Eid mane notun ki...",Joy
446,447,Eber tahola k sobai tel lagai ... . Ajob s...,Joy
447,448,Mojar chilo . koto manusher jhogra hoye gelo ...,Joy
448,449,apnake ekhon abp ananda te dekte paina keno,Joy


In [ ]:
df['label'].value_counts()

Joy         349
Sadness      50
Neutral      27
Disgust      21
Anger         2
Surprise      1
Name: label, dtype: int64

In [ ]:
 df.to_csv(folder_path +'becodemixed_svm.tsv',sep = '\t',index = False)

In [ ]:
print_metrices(actual_labels['label'],df['label'])

[[  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [ 35  62   3 160  81  71  25]
 [  0   0   0   1   0   1   0]
 [  1   0   0   0   0  10   0]
 [  0   0   0   0   0   0   0]]
              precision    recall  f1-score   support

       Anger       0.00      0.00      0.00         0
     Disgust       0.00      0.00      0.00         0
        Fear       0.00      0.00      0.00         0
         Joy       0.99      0.37      0.54       437
     Neutral       0.00      0.00      0.00         2
     Sadness       0.12      0.91      0.22        11
    Surprise       0.00      0.00      0.00         0

    accuracy                           0.38       450
   macro avg       0.16      0.18      0.11       450
weighted avg       0.97      0.38      0.52       450

Accuracy :  0.37777777777777777
Precison :  0.2966507177033493
Recall :  0.37777777777777777
F1 :  0.2306405661244371


## MNB

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb =   MultinomialNB(alpha=1.5)#alpha : float, default=1.0 additive (Laplace/Lidstone) smoothing parameter (0 for no smoothing).
nb.fit(tfidfvectorized_train,train_data['enc_label'])
y_pred = nb.predict(tfidfvectorized_valid)
print_metrices(y_pred,valid_data['enc_label'])

[[148   1   0   0   0   0   4]
 [ 72   5   0   0   0   0   0]
 [ 32   0   0   0   0   0   0]
 [ 78   0   0   0   0   0   0]
 [ 29   1   0   0   0   0   0]
 [  3   0   0   0   0   0   0]
 [ 71   0   0   0   0   0   7]]
              precision    recall  f1-score   support

           0       0.34      0.97      0.51       153
           1       0.71      0.06      0.12        77
           2       0.00      0.00      0.00        32
           3       0.00      0.00      0.00        78
           4       0.00      0.00      0.00        30
           5       0.00      0.00      0.00         3
           6       0.64      0.09      0.16        78

    accuracy                           0.35       451
   macro avg       0.24      0.16      0.11       451
weighted avg       0.35      0.35      0.22       451

Accuracy :  0.35476718403547675
Precison :  0.9319099737125041
Recall :  0.35476718403547675
F1 :  0.4906438890498295


In [ ]:
test_pred = nb.predict(tfidfvectorized_test).tolist()
test_data['id'] = test_data.index

In [ ]:
df = pd.DataFrame(test_data['id'])
# df['text'] = test_data['Text']
df['label'] = test_pred
df['label'] = df['label'].apply(actual_label)
df

,id,label
0,0,Joy
1,1,Joy
2,2,Joy
3,3,Joy
4,4,Joy
...,...,...
445,445,Joy
446,446,Joy
447,447,Joy
448,448,Joy


In [ ]:
df['label'].value_counts()

Joy        437
Sadness     11
Neutral      2
Name: label, dtype: int64

In [ ]:
print_metrices(actual_labels['label'],df['label'])

[[  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [ 35  62   3 160  81  71  25]
 [  0   0   0   1   0   1   0]
 [  1   0   0   0   0  10   0]
 [  0   0   0   0   0   0   0]]
              precision    recall  f1-score   support

       Anger       0.00      0.00      0.00         0
     Disgust       0.00      0.00      0.00         0
        Fear       0.00      0.00      0.00         0
         Joy       0.99      0.37      0.54       437
     Neutral       0.00      0.00      0.00         2
     Sadness       0.12      0.91      0.22        11
    Surprise       0.00      0.00      0.00         0

    accuracy                           0.38       450
   macro avg       0.16      0.18      0.11       450
weighted avg       0.97      0.38      0.52       450

Accuracy :  0.37777777777777777
Precison :  0.2966507177033493
Recall :  0.37777777777777777
F1 :  0.2306405661244371


In [ ]:
df.to_csv(folder_path +'becodemixed_mnbfinal.tsv',sep = '\t',index = False)

##Deep Learning

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import regularizers
from tensorflow.keras.layers import LSTM,GRU
from tensorflow.keras.models import load_model
print("Tensorflow Version:--",tf.version.VERSION)
print("Keras Version:---",keras.__version__)

Tensorflow Version:-- 2.8.0
Keras Version:--- 2.8.0


###Tokenization

In [ ]:
tokenizer = Tokenizer(num_words = 10000,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n-', 
                      split=' ', char_level=False, oov_token='<oov>', document_count=0)
tokenizer.fit_on_texts(train_data['Text'])  
vocab_size = len(tokenizer.word_index)+1
vocab_size

8670

## what is thattttttt

1.def def pad_sequences(sequences, maxlen=None, dtype='int32', padding='pre', truncating='pre', value=0.0)
Pads sequences to the same length.

This function transforms a list (of length num_samples) of sequences (lists of integers) into a 2D Numpy array of shape (num_samples, num_timesteps). num_timesteps is either the maxlen argument if provided, or the length of the longest sequence in the list.
2.Public API for tf.keras.preprocessing.sequence namespace.



In [ ]:
max_len = 300
# Training Sequences
train_sequences = tokenizer.texts_to_sequences(train_data['Text']) #def texts_to_sequences(texts)Transforms each text in texts to a sequence of integers.
print(len(tokenizer.word_index))
train_pad_sequences =  keras.preprocessing.sequence.pad_sequences(train_sequences, value=0.0, padding='post', maxlen= max_len)
print("Number of Training Sequences :" ,train_pad_sequences.shape)

# Validation Sequences
validation_sequences = tokenizer.texts_to_sequences(valid_data['Text'])
validation_pad_sequences =  keras.preprocessing.sequence.pad_sequences(validation_sequences, value=0.0, padding='post', maxlen= max_len)
print("Number of Validation Sequences :" ,validation_pad_sequences.shape)

8669
Number of Training Sequences : (2106, 300)
Number of Validation Sequences : (451, 300)


In [ ]:
# Test Sequences
test_sequences = tokenizer.texts_to_sequences(test_data['Text'])
test_pad_sequences =  keras.preprocessing.sequence.pad_sequences(test_sequences, value=0.0, padding='post', maxlen= max_len)
print("Number of Test Sequences :" ,test_pad_sequences.shape)

Number of Test Sequences : (450, 300)


In [ ]:

# Estimate class weights for unbalanced datasets.

# Parameters

# class_weight : dict, 'balanced' or None
#     If 'balanced', class weights will be given by
# n_samples / (n_classes * np.bincount(y)).
# If a dictionary is given, keys are classes and values
# are corresponding class weights.
# If None is given, the class weights will be uniform.

# classes : ndarray

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced'
                                               ,classes =np.unique(train_data['enc_label'])
                                               ,y=train_data['enc_label'])

weight = {i : class_weights[i] for i in range(7)}

In [ ]:
weight

{0: 0.42077922077922075,
 1: 0.7635968092820885,
 2: 2.4660421545667446,
 3: 0.9673863114377583,
 4: 2.060665362035225,
 5: 10.374384236453203,
 6: 0.7734116782959971}

In [ ]:
# def def __init__(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', save_freq='epoch', options=None, **kwargs)
# Callback to save the Keras model or model weights at some frequency.

# ModelCheckpoint callback is used in conjunction with training using model.fit() to save a model or weights (in a checkpoint file) at some interval, so the model or weights can be loaded later to continue the training from the state saved.

# A few options this callback provides include:

# Whether to only keep the model that has achieved the "best performance" so far, or whether to save the model at the end of every epoch regardless of performance.
# Definition of 'best'; which quantity to monitor and whether it should be maximized or minimized.
# The frequency it should save at. Currently, the callback supports saving at the end of every epoch, or after a fixed number of training batches.
# Whether only weights are saved, or the whole model is saved.
# Note: If you get WARNING:tensorflow:Can save best model only with <name> available, skipping see the description of the monitor argument for details on how to get this right.


# def def __init__(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None, **kwargs)
# Turns positive integers (indexes) into dense vectors of fixed size.

# e.g. [[4], [20]] -> [[0.25, 0.1], [0.6, -0.2]]

# This layer can only be used as the first layer in a model.

# Example:

# >>> model = tf.keras.Sequential()
# >>> model.add(tf.keras.layers.Embedding(1000, 64, input_length=10))
# >>> # The model will take as input an integer matrix of size (batch,
# >>> # input_length), and the largest integer (i.e. word index) in the input
# >>> # should be no larger than 999 (vocabulary size).
# >>> # Now model.output_shape is (None, 10, 64), where `None` is the batch
# >>> # dimension.

##cnn

In [ ]:
num_classes = 7

accuracy_threshold = 0.99

class myCallback(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
      if(logs.get('accuracy')>accuracy_threshold):
        print("\nReached %2.2f%% accuracy so we will stop trianing" % (accuracy_threshold*100))
        self.model.stop_training = True

acc_callback = myCallback()
# Saved the Best Model
filepath = folder_path+"becodemixed_cnn.h5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=2, save_best_only=True, 
                                             save_weights_only=False, mode='max')
# callback list
callback_list = [acc_callback, checkpoint] 
model = tf.keras.models.Sequential([
tf.keras.layers.Embedding(10000, 100, input_length = max_len),
# def def __init__(filters, kernel_size, strides=1, padding='valid', data_format='channels_last', dilation_rate=1, groups=1, activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None, **kwargs)
# 1D convolution layer (e.g. temporal convolution).
tf.keras.layers.Conv1D(128, 5, activation='relu'),
tf.keras.layers.MaxPooling1D(pool_size=2),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(11 , activation='softmax')])

model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 300, 100)          1000000   
                                                                 
 conv1d_2 (Conv1D)           (None, 296, 128)          64128     
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 148, 128)         0         
 1D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 18944)             0         
                                                                 
 dense_2 (Dense)             (None, 11)                208395    
                                                                 
Total params: 1,272,523
Trainable params: 1,272,523
Non-trainable params: 0
____________________________________________

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.003),loss='sparse_categorical_crossentropy', metrics=['accuracy'])
   
model.fit(train_pad_sequences,train_data['enc_label'], epochs=30, batch_size = 32, validation_split = 0.2,verbose =1,callbacks = callback_list,class_weight = weight)

Epoch 1/30
50/53 [===========================>..] - ETA: 0s - loss: 2.1023 - accuracy: 0.1213
Epoch 1: val_accuracy improved from -inf to 0.15640, saving model to /content/drive/MyDrive/model_imp_whole/becodemixed_cnn.h5
53/53 [==============================] - 1s 11ms/step - loss: 2.0777 - accuracy: 0.1211 - val_loss: 1.8998 - val_accuracy: 0.1564
Epoch 2/30
48/53 [==========================>...] - ETA: 0s - loss: 1.5262 - accuracy: 0.4512
Epoch 2: val_accuracy improved from 0.15640 to 0.32464, saving model to /content/drive/MyDrive/model_imp_whole/becodemixed_cnn.h5
53/53 [==============================] - 0s 8ms/step - loss: 1.5102 - accuracy: 0.4578 - val_loss: 1.7799 - val_accuracy: 0.3246
Epoch 3/30
52/53 [============================>.] - ETA: 0s - loss: 0.3357 - accuracy: 0.8990
Epoch 3: val_accuracy improved from 0.32464 to 0.42417, saving model to /content/drive/MyDrive/model_imp_whole/becodemixed_cnn.h5
53/53 [==============================] - 0s 9ms/step - loss: 0.3338 - ac

In [ ]:
# Load the saved model
model = load_model(folder_path+'becodemixed_cnn.h5')
# prediction
y_pred = np.argmax(model.predict(validation_pad_sequences), axis=-1)#numpy.argmax(a, axis=None, out=None, *, keepdims=<no value>)Returns the indices of the maximum values along an axis.
print_metrices(valid_data['enc_label'],y_pred)


[[103  22  14  24  10   2  18]
 [ 27  39   7  28  11   1  17]
 [  1   1   3   1   0   0   1]
 [  5   5   2  12   3   0   7]
 [  0   0   2   3   2   0   1]
 [  0   1   0   1   0   0   0]
 [ 17   9   4   9   4   0  34]]
              precision    recall  f1-score   support

           0       0.67      0.53      0.60       193
           1       0.51      0.30      0.38       130
           2       0.09      0.43      0.15         7
           3       0.15      0.35      0.21        34
           4       0.07      0.25      0.11         8
           5       0.00      0.00      0.00         2
           6       0.44      0.44      0.44        77

    accuracy                           0.43       451
   macro avg       0.28      0.33      0.27       451
weighted avg       0.52      0.43      0.46       451

Accuracy :  0.4279379157427938
Precison :  0.41671420324053804
Recall :  0.4279379157427938
F1 :  0.39716538917000127


In [ ]:
test_pred = np.argmax(model.predict(test_pad_sequences), axis=-1).tolist()
# test_pred
test_data['id'] = test_data.index + 1
df = pd.DataFrame(test_data['id'])
# df['text'] = test_data['Text']
df['label'] = test_pred
df['label'] = df['label'].apply(actual_label)
df

,id,label
0,1,Sadness
1,2,Joy
2,3,Joy
3,4,Disgust
4,5,Joy
...,...,...
445,446,Joy
446,447,Joy
447,448,Joy
448,449,Sadness


In [ ]:
df['label'].value_counts()

Joy         226
Sadness     114
Neutral      60
Disgust      33
Anger        11
Surprise      5
Fear          1
Name: label, dtype: int64

In [ ]:
print_metrices(actual_labels['label'],df['label'])

[[  3   0   0   1   2   2   3]
 [  3  17   0   3   6   2   2]
 [  0   0   0   0   1   0   0]
 [  8  20   1 124  30  29  14]
 [  9   6   1  15  24   3   2]
 [ 13  17   1  18  16  46   3]
 [  0   2   0   0   2   0   1]]
              precision    recall  f1-score   support

       Anger       0.08      0.27      0.13        11
     Disgust       0.27      0.52      0.36        33
        Fear       0.00      0.00      0.00         1
         Joy       0.77      0.55      0.64       226
     Neutral       0.30      0.40      0.34        60
     Sadness       0.56      0.40      0.47       114
    Surprise       0.04      0.20      0.07         5

    accuracy                           0.48       450
   macro avg       0.29      0.33      0.29       450
weighted avg       0.59      0.48      0.52       450

Accuracy :  0.4777777777777778
Precison :  0.44573684053702683
Recall :  0.4777777777777778
F1 :  0.4393095014448645


In [ ]:
df.to_csv(folder_path +'becodemixed_cnnfinal.tsv',sep = '\t',index = False)

## lstm

In [ ]:
num_classes = 7
from keras.layers.embeddings import Embedding
accuracy_threshold = 0.99

class myCallback(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
      if(logs.get('accuracy')>accuracy_threshold):
        print("\nReached %2.2f%% accuracy so we will stop trianing" % (accuracy_threshold*100))
        self.model.stop_training = True

acc_callback = myCallback()
# Saved the Best Model
filepath = folder_path+"becodemixed_LSTM.h5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=2, save_best_only=True, 
                                             save_weights_only=False, mode='max')
# callback list
callback_list = [acc_callback, checkpoint] 
# lstm
model = tf.keras.models.Sequential([
tf.keras.layers.Embedding(10000, 100, input_length = max_len),

tf.keras.layers.LSTM(units = 128,activation='tanh',return_sequences=True,dropout = 0.2),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(11 , activation='softmax')])

model.summary()



Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 300, 100)          1000000   
                                                                 
 lstm_1 (LSTM)               (None, 300, 128)          117248    
                                                                 
 flatten_4 (Flatten)         (None, 38400)             0         
                                                                 
 dense_4 (Dense)             (None, 11)                422411    
                                                                 
Total params: 1,539,659
Trainable params: 1,539,659
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.003),loss='sparse_categorical_crossentropy', metrics=['accuracy'])
   
model.fit(train_pad_sequences,train_data['enc_label'], epochs=10, batch_size = 32, validation_split = 0.2,verbose =1,callbacks = callback_list,class_weight = weight)

Epoch 1/10
52/53 [============================>.] - ETA: 0s - loss: 0.4130 - accuracy: 0.8684
Epoch 1: val_accuracy did not improve from 0.46682
53/53 [==============================] - 6s 51ms/step - loss: 0.4082 - accuracy: 0.8700 - val_loss: 2.9630 - val_accuracy: 0.4171
Epoch 2/10
53/53 [==============================] - ETA: 0s - loss: 0.0697 - accuracy: 0.9804
Epoch 2: val_accuracy did not improve from 0.46682
53/53 [==============================] - 2s 36ms/step - loss: 0.0697 - accuracy: 0.9804 - val_loss: 3.0302 - val_accuracy: 0.4147
Epoch 3/10
53/53 [==============================] - ETA: 0s - loss: 0.0489 - accuracy: 0.9863
Epoch 3: val_accuracy did not improve from 0.46682
53/53 [==============================] - 2s 35ms/step - loss: 0.0489 - accuracy: 0.9863 - val_loss: 3.1932 - val_accuracy: 0.3957
Epoch 4/10
53/53 [==============================] - ETA: 0s - loss: 0.0345 - accuracy: 0.9875
Epoch 4: val_accuracy did not improve from 0.46682
53/53 [=======================

In [ ]:
# Load the saved model
model = load_model(folder_path+'becodemixed_LSTM.h5')
# prediction
y_pred = np.argmax(model.predict(validation_pad_sequences), axis=-1)#numpy.argmax(a, axis=None, out=None, *, keepdims=<no value>)Returns the indices of the maximum values along an axis.
print_metrices(valid_data['enc_label'],y_pred)


[[105  27  14  23   9   2  37]
 [ 11  26   0  10   4   1   9]
 [ 12   0   7  11   2   0   5]
 [  4   6   6  18   6   0   7]
 [  7   9   3   8   7   0   6]
 [  0   1   0   1   0   0   1]
 [ 14   8   2   7   2   0  13]]
              precision    recall  f1-score   support

           0       0.69      0.48      0.57       217
           1       0.34      0.43      0.38        61
           2       0.22      0.19      0.20        37
           3       0.23      0.38      0.29        47
           4       0.23      0.17      0.20        40
           5       0.00      0.00      0.00         3
           6       0.17      0.28      0.21        46

    accuracy                           0.39       451
   macro avg       0.27      0.28      0.26       451
weighted avg       0.46      0.39      0.41       451

Accuracy :  0.3902439024390244
Precison :  0.37709934132177203
Recall :  0.3902439024390244
F1 :  0.3706517138011682


In [ ]:
test_pred = np.argmax(model.predict(test_pad_sequences), axis=-1).tolist()
# test_pred
test_data['id'] = test_data.index + 1
df = pd.DataFrame(test_data['id'])
# df['text'] = test_data['Text']
df['label'] = test_pred
df['label'] = df['label'].apply(actual_label)
df

,id,label
0,1,Joy
1,2,Joy
2,3,Joy
3,4,Disgust
4,5,Neutral
...,...,...
445,446,Joy
446,447,Anger
447,448,Joy
448,449,Anger


In [ ]:
df['label'].value_counts()

Joy         229
Disgust      58
Sadness      45
Anger        45
Surprise     43
Neutral      30
Name: label, dtype: int64

In [ ]:
print_metrices(actual_labels['label'],df['label'])

[[ 11   7   1   8  13   3   2]
 [  6  24   0   6  10   8   4]
 [  0   0   0   0   0   0   0]
 [  6  18   2 127  37  27  12]
 [  5   2   0   5   8   9   1]
 [  3   6   0   5   4  27   0]
 [  5   5   0  10   9   8   6]]
              precision    recall  f1-score   support

       Anger       0.31      0.24      0.27        45
     Disgust       0.39      0.41      0.40        58
        Fear       0.00      0.00      0.00         0
         Joy       0.79      0.55      0.65       229
     Neutral       0.10      0.27      0.14        30
     Sadness       0.33      0.60      0.43        45
    Surprise       0.24      0.14      0.18        43

    accuracy                           0.45       450
   macro avg       0.31      0.32      0.30       450
weighted avg       0.54      0.45      0.48       450

Accuracy :  0.45111111111111113
Precison :  0.4400705646973405
Recall :  0.45111111111111113
F1 :  0.4230839336622878


## bilstm

In [ ]:
num_classes = 7

accuracy_threshold = 0.99

class myCallback(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
      if(logs.get('accuracy')>accuracy_threshold):
        print("\nReached %2.2f%% accuracy so we will stop trianing" % (accuracy_threshold*100))
        self.model.stop_training = True

acc_callback = myCallback()
# Saved the Best Model
filepath = folder_path+"becodemixed_BiLSTM.h5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=2, save_best_only=True, 
                                             save_weights_only=False, mode='max')
# callback list
callback_list = [acc_callback, checkpoint] 
model = tf.keras.models.Sequential([
tf.keras.layers.Embedding(10000, 100, input_length = max_len),
#tf.keras.layers.Conv1D(128, 5, activation='relu'),
#tf.keras.layers.MaxPooling1D(5),
tf.keras.layers.Bidirectional(LSTM(units = 128,return_sequences=True,dropout = 0.2)),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(11 , activation='softmax')])

model.summary()


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 300, 100)          1000000   
                                                                 
 bidirectional_2 (Bidirectio  (None, 300, 256)         234496    
 nal)                                                            
                                                                 
 flatten_7 (Flatten)         (None, 76800)             0         
                                                                 
 dense_7 (Dense)             (None, 11)                844811    
                                                                 
Total params: 2,079,307
Trainable params: 2,079,307
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),loss='sparse_categorical_crossentropy', metrics=['accuracy'])
   
model.fit(train_pad_sequences,train_data['enc_label'], epochs=20, batch_size = 32, validation_split = 0.2,verbose =1,callbacks = callback_list,class_weight = weight)

Epoch 1/20
52/53 [============================>.] - ETA: 0s - loss: 2.0984 - accuracy: 0.1016
Epoch 1: val_accuracy improved from -inf to 0.04976, saving model to /content/drive/MyDrive/model_imp_whole/becodemixed_BiLSTM.h5
53/53 [==============================] - 6s 54ms/step - loss: 2.0937 - accuracy: 0.1010 - val_loss: 1.9435 - val_accuracy: 0.0498
Epoch 2/20
53/53 [==============================] - ETA: 0s - loss: 1.8109 - accuracy: 0.3432
Epoch 2: val_accuracy improved from 0.04976 to 0.30806, saving model to /content/drive/MyDrive/model_imp_whole/becodemixed_BiLSTM.h5
53/53 [==============================] - 2s 39ms/step - loss: 1.8109 - accuracy: 0.3432 - val_loss: 1.7760 - val_accuracy: 0.3081
Epoch 3/20
53/53 [==============================] - ETA: 0s - loss: 0.9058 - accuracy: 0.6906
Epoch 3: val_accuracy improved from 0.30806 to 0.38863, saving model to /content/drive/MyDrive/model_imp_whole/becodemixed_BiLSTM.h5
53/53 [==============================] - 2s 38ms/step - loss: 

In [ ]:
# Load the saved model
model = load_model(folder_path+'becodemixed_BiLSTM.h5')
# prediction
y_pred = np.argmax(model.predict(validation_pad_sequences), axis=-1)#numpy.argmax(a, axis=None, out=None, *, keepdims=<no value>)Returns the indices of the maximum values along an axis.
print_metrices(valid_data['enc_label'],y_pred)


[[99 16 13 12  3  1 19]
 [12 24  3 10  1  1  8]
 [ 7  2  5  3  1  1  4]
 [14 19  7 32 15  0 13]
 [ 7  6  1 13  6  0  4]
 [ 3  2  2  1  0  0  1]
 [11  8  1  7  4  0 29]]
              precision    recall  f1-score   support

           0       0.65      0.61      0.63       163
           1       0.31      0.41      0.35        59
           2       0.16      0.22      0.18        23
           3       0.41      0.32      0.36       100
           4       0.20      0.16      0.18        37
           5       0.00      0.00      0.00         9
           6       0.37      0.48      0.42        60

    accuracy                           0.43       451
   macro avg       0.30      0.31      0.30       451
weighted avg       0.44      0.43      0.43       451

Accuracy :  0.43237250554323725
Precison :  0.4406425739920145
Recall :  0.43237250554323725
F1 :  0.43251092651496054


### Test Set Prediction

In [ ]:
test_pred = np.argmax(model.predict(test_pad_sequences), axis=-1).tolist()
# test_pred

In [ ]:
test_data['id'] = test_data.index 
df = pd.DataFrame(test_data['id'])
# df['text'] = test_data['Text']
df['label'] = test_pred
df['label'] = df['label'].apply(actual_label)
df

,id,label
0,0,Sadness
1,1,Joy
2,2,Joy
3,3,Disgust
4,4,Neutral
...,...,...
445,445,Joy
446,446,Joy
447,447,Sadness
448,448,Sadness


In [ ]:
df['label'].value_counts()

Joy         179
Sadness     107
Disgust      92
Neutral      38
Anger        21
Surprise      7
Fear          6
Name: label, dtype: int64

In [ ]:
print_metrices(actual_labels['label'],df['label'])

[[  4   3   0   4   5   2   3]
 [ 13  31   1  14  16   7  10]
 [  0   0   0   2   4   0   0]
 [  5  13   2 105  31  15   8]
 [  1   1   0  16  13   5   2]
 [ 11  14   0  19  11  51   1]
 [  2   0   0   1   1   2   1]]
              precision    recall  f1-score   support

       Anger       0.11      0.19      0.14        21
     Disgust       0.50      0.34      0.40        92
        Fear       0.00      0.00      0.00         6
         Joy       0.65      0.59      0.62       179
     Neutral       0.16      0.34      0.22        38
     Sadness       0.62      0.48      0.54       107
    Surprise       0.04      0.14      0.06         7

    accuracy                           0.46       450
   macro avg       0.30      0.30      0.28       450
weighted avg       0.53      0.46      0.48       450

Accuracy :  0.45555555555555555
Precison :  0.42790190005298356
Recall :  0.45555555555555555
F1 :  0.42881954279142104


In [ ]:
df.to_csv(folder_path +'tamil_bilstmfinal3.tsv',sep ='\t',index = False)

In [ ]:
df.drop(labels=None, axis=0, index=None, columns=None, level=None, inplace=False, errors='raise')

ValueError: ignored

In [ ]:
df.to_csv(folder_path +'tamil_bilstmfinal2.tsv',sep = '\t',index = False)

In [ ]:
%%time
t =  pd.read_csv('/content/drive/MyDrive/submit/CUET16_run1xb.tsv')

t1 =  pd.read_csv('/content/drive/MyDrive/submit/CUET16_run2mlb.tsv')
t2 =  pd.read_csv('/content/drive/MyDrive/submit/CUET16_run3BL.tsv')

FileNotFoundError: ignored

In [ ]:

t2



# Data Statistics

In [ ]:
df=train_data 
df['word_count'] = df.cleaned.apply(lambda x: len(str(x).split()))
more_than_150 =  df['word_count'] > 150
df3 = df[more_than_150]
df3

In [ ]:
def char_length(row):
    return len(row.split())

In [ ]:
lables = ['Neutral' ,'Joy','Ambiguous','Trust','Disguist','Anger' ,'Anticipation','Sadness','Love','Surprise','Fear']

# Count number of unique words in each set
def dataset_statistics(dataset):
  for l in lables:
    word_list = [word.strip() for t in list(dataset[dataset[l]==1.0].cleaned) for word in t.strip().split()]
    print(f"\nClass Name:--> {l} ") 
    print("Number of Words:{}".format(len(word_list))) 
    print("Number of Unique Words:{}".format(len(np.unique(word_list)))) 
    print("Maximum post length: {}".format(max(dataset[dataset[l]==1.0].cleaned.apply(char_length))))
    print("Average Words in texts: {}".format(np.mean(dataset[dataset[l]==1.0].cleaned.apply(char_length))))
    

In [ ]:
train_data.head(2)

In [ ]:
dataset_statistics(train_data)

# Data Statistics

In [ ]:
df=train_data 
df['word_count'] = df.Text.apply(lambda x: len(str(x).split()))
more_than_150 =  df['word_count'] > 170
df3 = df[more_than_150]
df3

In [ ]:
def char_length(row):
    return len(row.split())

In [ ]:
lables = ['Neutral' ,'Joy','Ambiguous','Trust','Disguist','Anger' ,'Anticipation','Sadness','Love','Surprise','Fear']

# Count number of unique words in each set
def dataset_statistics(dataset):
  for l in lables:
    word_list = [word.strip() for t in list(dataset[dataset[l]==1.0].Text) for word in t.strip().split()]
    print(f"\nClass Name:--> {l} ") 
    print("Number of Words:{}".format(len(word_list))) 
    print("Number of Unique Words:{}".format(len(np.unique(word_list)))) 
    print("Maximum post length: {}".format(max(dataset[dataset[l]==1.0].Text.apply(char_length))))
    print("Average Words in texts: {}".format(np.mean(dataset[dataset[l]==1.0].Text.apply(char_length))))
    

In [ ]:
train_data.head(2)

In [ ]:
dataset_statistics(train_data)